In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
import elasticsearch
import json


In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [11]:
from elasticsearch import Elasticsearch

In [14]:
es_client = Elasticsearch("http://localhost:9200")

In [15]:
es_client.info()

ObjectApiResponse({'name': '99a45062d532', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ryH-LOA3SJ6mQv-esWU3JQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_26206/1194577603.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [17]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [19]:
query = "I just discovered the course. Can I still join it?"

In [24]:
query = "How do I run kafka?"

In [25]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type":"best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [26]:
response = es_client.search(index=index_name, body=search_query)

/tmp/ipykernel_26206/449040615.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [27]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'lWqWb5ABgQclO23MpxeA',
  '_score': 30.447786,
  '_source': {'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
   'section': 'Workshop 1 - dlthub',
   'question': 'How do I install the necessary dependencies to run the code?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'fmqWb5ABgQclO23MpRdL',
  '_score': 26.374605,
  '_source': {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
   'section': 'Module 6: streaming with kafka',
   'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-

In [28]:
query = "How do I execute a command in a running docker container?"

In [32]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type":"best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [33]:
response = es_client.search(index=index_name, body=search_query)

/tmp/ipykernel_26206/449040615.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [38]:
response

ObjectApiResponse({'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions', '_id': 'VWqWb5ABgQclO23Muhgv', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'dGqWb5ABgQclO23MvRgs', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo c

In [49]:
response['hits']['hits'][0]

{'_index': 'course-questions',
 '_id': 'VWqWb5ABgQclO23Muhgv',
 '_score': 84.050095,
 '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'}}

In [52]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

context_template = """
Q: {question}
A: {text}
""".strip()

context = ""
    
for doc in response['hits']['hits']:
    context = context + f"Q: {doc['_source']['question']}\nA: {doc['_source']['text']}".strip() + "\n\n"

# for doc in response:
#     context = context_template.format(question={doc['question']}, text={doc['text']}) + "\n\n"
    
prompt = prompt_template.format(question=query, context=context).strip()

In [53]:
len(prompt)

2715

In [59]:
encoding = tiktoken.encoding_for_model("gpt-4o")
encoding

<Encoding 'o200k_base'>

In [64]:
import tiktoken

In [67]:
tokenizer = tiktoken.get_encoding("o200k_base")

In [68]:
# tokenizer = tiktoken.TextEncoder(encoding=encoding)
tokens = tokenizer.encode(prompt)
len(tokens)

622